In [7]:
import os
import time 
import datetime

def print_table(data: dict[str, list[float]], precision: int = 4):
    # Convert to list of keys and find number of rows
    headers = list(data.keys())
    headers = ["index"] + headers        
    n_rows = len(next(iter(data.values())))
    data["index"] = list(range(1, n_rows+1))

    # Format floats to desired precision
    fmt = f"{{:.{precision}f}}"

    # Compute column widths
    col_widths = []
    for h in headers:
        values = data[h]
        max_val_len = max(len(fmt.format(v)) for v in values)
        col_widths.append(max(len(h), max_val_len))

    # Build header row
    header_row = " | ".join(f"{h:>{w}}" for h, w in zip(headers, col_widths))
    separator = "-+-".join("-" * w for w in col_widths)

    # Print table
    print(header_row)
    print(separator)
    for i in range(n_rows):
        row = " | ".join(fmt.format(data[h][i]).rjust(w) for h, w in zip(headers, col_widths))
        print(row)

In [8]:
def dt_to_intkey(date_str: str, time_str: str) -> float:
    dt = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y_%m_%d %H_%M_%S")
    return dt.timestamp()

In [10]:
data_dir = "../../logs/results"
res_files_unfiltered = [f for f in os.listdir(data_dir) if f.endswith(".log")]
res_files_unfiltered.sort()
# print(*res_files, sep="\n")
dates, times = zip(*(s.split("__")[:2] for s in res_files_unfiltered))
creation_times = {
    # key is date_time string converted to int/float for sorting
    index: dt_to_intkey(date, time)
    for index, (date, time) in enumerate(zip(dates, times))
}
only_after = dt_to_intkey("2025_10_22", "00_00_00")
res_files = [
    res_files_unfiltered[index]
    for index, creation_time in creation_times.items()
    if creation_time >= only_after
]
# print(*res_files, sep="\n")
print(len(res_files), "files found after filtering.")

48 files found after filtering.


In [11]:
results_y_co_cg = [{} for _ in range(23)]
results_rgb = [{} for _ in range(23)]

for f in res_files:
    with open(os.path.join(data_dir, f), "r") as infile:
        lines = infile.readlines()
        ycocg = True
        for line in lines:
            if "RGB" in line:
                ycocg = False
            if "Final results after quantization" in line:
                parts = (
                    line[len("Final results after quantization:") :]
                    .strip()
                    .split(", ")
                )
                for part in parts:
                    key, value = part.split(": ")
                    if ycocg:
                        results_y_co_cg[
                            int(
                                f.split("_")[-1][len("kodim") :][: -len(".log")]
                            )
                            - 1
                        ][key] = float(value)
                    else:
                        results_rgb[
                            int(
                                f.split("_")[-1][len("kodim") :][: -len(".log")]
                            )
                            - 1
                        ][key] = float(value)


def list_dict_to_dict_list(
    lst: list[dict[str, float]],
) -> dict[str, list[float]]:
    if not lst:
        return {}
    keys = lst[0].keys()
    dict_list = {key: [] for key in keys}
    for d in lst:
        for key in keys:
            dict_list[key].append(d[key])
    return dict_list


results_y_co_cg = list_dict_to_dict_list(results_y_co_cg)
results_rgb = list_dict_to_dict_list(results_rgb)

print("YCoCg Results:")
print_table(results_y_co_cg)
for key, values in results_y_co_cg.items():
    avg = sum(values) / len(values) if values else 0
    print(f"{key}: {avg:.4f}")

print("RGB Results:")
print_table(results_rgb)
for key, values in results_rgb.items():
    avg = sum(values) / len(values) if values else 0
    print(f"{key}: {avg:.4f}")

YCoCg Results:
  index |   Loss | Rate NN | Rate Latent | Rate Img
--------+--------+---------+-------------+---------
 1.0000 | 2.5045 |  0.0089 |      0.0132 |   2.4824
 2.0000 | 1.9950 |  0.0042 |      0.0010 |   1.9898
 3.0000 | 1.8644 |  0.0093 |      0.0480 |   1.8070
 4.0000 | 2.0343 |  0.0107 |      0.1141 |   1.9095
 5.0000 | 2.4676 |  0.0067 |      0.0307 |   2.4302
 6.0000 | 2.5335 |  0.0037 |      0.0025 |   2.5272
 7.0000 | 2.0888 |  0.0096 |      0.0608 |   2.0183
 8.0000 | 2.5839 |  0.0098 |      0.0207 |   2.5534
 9.0000 | 1.9717 |  0.0090 |      0.0318 |   1.9309
10.0000 | 2.2121 |  0.0067 |      0.0251 |   2.1803
11.0000 | 2.0702 |  0.0030 |      0.0118 |   2.0554
12.0000 | 2.0839 |  0.0082 |      0.0295 |   2.0462
13.0000 | 2.7715 |  0.0095 |      0.0676 |   2.6944
14.0000 | 2.2566 |  0.0101 |      0.1020 |   2.1445
15.0000 | 1.8814 |  0.0092 |      0.0204 |   1.8517
16.0000 | 1.9587 |  0.0076 |      0.0238 |   1.9273
17.0000 | 1.9699 |  0.0035 |      0.0015 |   1.96